In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from langchain_chroma import Chroma

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from langchain_core.prompts.chat import ChatPromptTemplate

from huggingface_hub import notebook_login
notebook_login()

In [18]:
model_id = "meta-llama/Llama-2-7b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.50it/s]


In [12]:
CHROMA_PATH = "chroma_db"

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [3]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\denis\anaconda3\envs\qenv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [4]:
db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)

In [5]:
query_text = 'Why is numpy fast?'

In [8]:
results = db.similarity_search_with_relevance_scores(query_text)

In [9]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])

In [24]:
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [20]:
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
    (rotary_emb)

In [21]:
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

output = model.generate(input_ids)

response = tokenizer.decode(output[0], skip_special_tokens=True)

In [22]:
print(response)

Human: 
Answer the question based only on the following context:

CHAPTER
ONE
SETTING UP
1.1 What is NumPy?
NumPy is the fundamental package for scientiﬁc computing in Python. It is a Python library that provides a multidi-
mensional array object, various derived objects (such as masked arrays and matrices), and an assortment of routines for
fast operations on arrays, including mathematical, logical, shape manipulation, sorting, selecting, I/O, discrete Fourier
transforms, basic linear algebra, basic statistical operations, random simulation and much more.

---

Python. Indeed, the NumPy idiom is even simpler! This last example illustrates two of NumPy’s features which are
the basis of much of its power: vectorization and broadcasting.
1.1.1 Why is NumPy Fast?
Vectorization describes the absence of any explicit looping, indexing, etc., in the code - these things are taking place,
of course, just “behind the scenes” in optimized, pre-compiled C code. Vectorized code has many advantages,